#Install

In [2]:
!pip install hugsvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 24.8 MB/s eta 0:00:00


#Imports

In [3]:
#%%
# source: https://medium.com/@yanis.labrak/how-to-train-a-custom-vision-transformer-vit-image-classifier-to-help-endoscopists-in-under-5-min-2e7e4110a353
import pandas
#%%
#%% Packages
from hugsvision.dataio.VisionDataset import VisionDataset
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer
from transformers import ViTFeatureExtractor, ViTForImageClassification
from hugsvision.inference.VisionClassifierInference import VisionClassifierInference

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

/usr/local/lib/python3.9/dist-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [4]:
#%% packages
import requests
import urllib.request
from pathlib import Path

#%%
def get_dog_images(breed, count=100, type='train'):
    available_breeds = ['affenpinscher', 'african', 'airedale', 'akita', 'appenzeller', 'australian', 'basenji', 'beagle', 'bluetick', 'borzoi', 'bouvier', 'boxer', 'brabancon', 'briard', 'buhund', 'bulldog', 'bullterrier', 'cattledog', 'chihuahua', 'chow', 'clumber', 'cockapoo', 'collie', 'coonhound', 'corgi', 'cotondetulear', 'dachshund', 'dalmatian', 'dane', 'deerhound', 'dhole', 'dingo', 'doberman', 'elkhound', 'entlebucher', 'eskimo', 'finnish', 'frise', 'germanshepherd', 'greyhound', 'groenendael', 'havanese', 'hound', 'husky', 'keeshond', 'kelpie', 'komondor', 'kuvasz', 'labradoodle', 'labrador', 'leonberg', 'lhasa', 'malamute', 'malinois', 'maltese', 'mastiff', 'mexicanhairless', 'mix', 'mountain', 'newfoundland', 'otterhound', 'ovcharka', 'papillon', 'pekinese', 'pembroke', 'pinscher', 'pitbull', 'pointer', 'pomeranian', 'poodle', 'pug', 'puggle', 'pyrenees', 'redbone', 'retriever', 'ridgeback', 'rottweiler', 'saluki', 'samoyed', 'schipperke', 'schnauzer', 'setter', 'sheepdog', 'shiba', 'shihtzu', 'spaniel', 'springer', 'stbernard', 'terrier', 'tervuren', 'vizsla', 'waterdog', 'weimaraner', 'whippet', 'wolfhound']
    for i in range(count):
        if breed not in available_breeds:
            raise ValueError('Breed not available')
        else:
            API = f'https://dog.ceo/api/breed/{breed}/images/random'
        
        img_url = requests.get(API).json()['message']
        # create folder if necessary
        breed_path = f"{type}/{breed}"
        Path(breed_path).mkdir(parents=True, exist_ok=True)
        
        # download an image and save it in folder
        img = urllib.request.urlretrieve(img_url, f"{breed_path}/{breed}_{i}.jpg")
    return None

def get_breeds():
    API = 'https://dog.ceo/api/breeds/list/all'
    breeds = requests.get(API).json()['message']
    return breeds.keys()

# %% get train images
get_dog_images('affenpinscher')
get_dog_images('corgi')
get_dog_images('akita')
#%% get test images
get_dog_images('affenpinscher', type='test', count=20)
get_dog_images('corgi', type='test', count=20)
get_dog_images('akita', type='test', count=20)

# %%

In [5]:
#%% data prep
train, val, id2label, label2id = VisionDataset.fromImageFolder(
    "/content/train",
    test_ratio   = 0.1,
    balanced     = True,
    augmentation = True, 
    torch_vision = False
)

Split Datasets...
Balance train dataset...
The less represented label in train as 100 occurrences
Size of train after balancing is 300
Training Dataset Elements:  270
+---------+---------------+-------+-------+-------+
| Dataset | affenpinscher | akita | corgi | Total |
+---------+---------------+-------+-------+-------+
|  Train  |      84       |  92   |  94   |  270  |
|  Test   |      16       |   8   |   6   |  30   |
+---------+---------------+-------+-------+-------+


In [6]:
#%%
huggingface_model = 'google/vit-base-patch16-224-in21k'

trainer = VisionClassifierTrainer(
	model_name   = "MyDogClassifier",
	train        = train,
	test         = val,
	output_dir   = "./out/",
	max_epochs   = 20,
	batch_size   = 4, 
	lr	     = 2e-5,
	fp16	     = True,
	model = ViTForImageClassification.from_pretrained(
	    huggingface_model,
	    num_labels = len(label2id),
	    label2id   = label2id,
	    id2label   = id2label
	),
	feature_extractor = ViTFeatureExtractor.from_pretrained(
		huggingface_model,
	),
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


{'0': 'affenpinscher', '1': 'akita', '2': 'corgi'}
{'affenpinscher': '0', 'akita': '1', 'corgi': '2'}


Using cuda_amp half precision backend
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 270
  Num Epochs = 20
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1360
  Number of trainable parameters = 85800963


Trainer builded!
Start Training!


Epoch,Training Loss,Validation Loss
1,No log,0.317261
2,No log,0.093121
3,No log,0.055356
4,No log,0.041668
5,No log,0.034142
6,No log,0.029416
7,No log,0.026138
8,0.147000,0.023629
9,0.147000,0.021774
10,0.147000,0.020249


***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 30
  Batch size = 4
***** Running Evaluation *****
  Num examples = 

Model saved at: ./out/MYDOGCLASSIFIER/20_2023-03-10-12-24-01


In [10]:
#%% Model Evaluation
y_true, y_pred = trainer.evaluate_f1_score()

#%%
cm = confusion_matrix(y_true, y_pred)
labels = list(label2id.keys())
df_cm = pd.DataFrame(cm, index = labels, columns = labels)

# plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True, annot_kws={"size": 8}, fmt="")
plt.savefig("./conf_matrix_1.jpg")

# %% Inference
import os.path
path = "/content/out/MYDOGCLASSIFIER/20_2023-03-10-12-24-01/model/"
img  = "./test/affenpinscher/affenpinscher_0.jpg"

classifier = VisionClassifierInference(
    feature_extractor = ViTFeatureExtractor.from_pretrained('/content/out/MYDOGCLASSIFIER/20_2023-03-10-12-24-01/feature_extractor/'),
    model = ViTForImageClassification.from_pretrained(path),
)

label = classifier.predict(img_path=img)
print("Predicted class:", label)


100%|██████████| 30/30 [00:01<00:00, 25.53it/s]


               precision    recall  f1-score   support

affenpinscher     1.0000    1.0000    1.0000        16
        akita     1.0000    1.0000    1.0000         8
        corgi     1.0000    1.0000    1.0000         6

     accuracy                         1.0000        30
    macro avg     1.0000    1.0000    1.0000        30
 weighted avg     1.0000    1.0000    1.0000        30

Logs saved at: ./out/MYDOGCLASSIFIER/20_2023-03-10-12-24-01


loading configuration file /content/out/MYDOGCLASSIFIER/20_2023-03-10-12-24-01/feature_extractor/preprocessor_config.json
/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Image processor ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

loading configuration file /content/out/MYDOGCLASSIFIER/20_2023-03-10-12-24-01/model/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dr

Model loaded!
Predicted class: affenpinscher


In [11]:
# %% Test dataset
test, _, id2label, label2id = VisionDataset.fromImageFolder(
    "/content/test/",
    test_ratio   = 0,
    balanced     = True,
    augmentation = True, 
    torch_vision = False
)

classifier.predict(img)
# %% 
import glob
test_files = [f for f in glob.glob("/content/test/**/**", recursive=True) if os.path.isfile(f)]
# %%
for i in range(len(test_files)):
	print(f"{test_files[i]}")
	print(f"predicted: {classifier.predict(test_files[i])}")
# %%

Split Datasets...
Balance train dataset...
The less represented label in train as 20 occurrences
Size of train after balancing is 60
Training Dataset Elements:  60
+---------+---------------+-------+-------+-------+
| Dataset | affenpinscher | akita | corgi | Total |
+---------+---------------+-------+-------+-------+
|  Train  |      20       |  20   |  20   |  60   |
|  Test   |       0       |   0   |   0   |   0   |
+---------+---------------+-------+-------+-------+
/content/test/affenpinscher/affenpinscher_5.jpg
predicted: affenpinscher
/content/test/affenpinscher/affenpinscher_16.jpg
predicted: affenpinscher
/content/test/affenpinscher/affenpinscher_13.jpg
predicted: affenpinscher
/content/test/affenpinscher/affenpinscher_11.jpg
predicted: affenpinscher
/content/test/affenpinscher/affenpinscher_6.jpg
predicted: affenpinscher
/content/test/affenpinscher/affenpinscher_17.jpg
predicted: affenpinscher
/content/test/affenpinscher/affenpinscher_4.jpg
predicted: affenpinscher
/content/